### Setting things up

Make sure your cluster is started (start the engines on the dashboard, if necessary) and then execute the following:

In [ ]:
%pylab inline
import uuid
import pickle
import datetime

import starlab
from IPython.parallel import Client
rc = Client(profile="default")
len(rc.ids)

In [ ]:
lbv = rc.load_balanced_view()

We're now ready to generate an ensemble's worth of data.  Here's the function we'll use to do it.

In [ ]:
def run_ensemble(rc=rc, nruns=10, nstars=100, kingmodel=True, **kwargs):
    starttime = datetime.datetime.now()
    print "Starting at ", starttime
    lbv = rc.load_balanced_view()
    runs = [starlab.Run(nstars=nstars, kingmodel=kingmodel, **kwargs)]*nruns

    print "Launching %d runs" % nruns
    results = lbv.map(starlab.single_run, runs)
    results.wait()
    
    ensemble_uuid = uuid.uuid1()
    ensemble_results_file_name = str(ensemble_uuid)+".ensemble"
    ensemble_results_file = open(ensemble_results_file_name, "wb")

    print "Storing results in %s" %ensemble_results_file_name
    resultlist = results[:]
    pickle.dump(resultlist, ensemble_results_file)
    
    endtime = datetime.datetime.now()
    print "Finished at ", endtime
    print "Elapsed time: ", endtime - starttime 

You'll note that the default is to run a small ensemble (10 runs) of small clusters (100 stars each) of King models.

In [ ]:
run_ensemble(kingmodel=False)

The results for each run in the ensemble are stored in a (very much improvised) data warehouse, and the information of how to retrieve the necessary information is stored in a file with the extension ".ensemble"  The file name is actually a universally unique identifier (UUID), which isn't very meaningful by itself:

In [ ]:
ls *.ensemble

In [ ]:
more 1fd770ec-91cb-11e3-be2b-90e2ba0993e2.ensemble

But, we can always go to it later and figure out what is inside through the magic of introspection.  Here I open the ensemble, load up the results, and make an animation of run number 8 within the ensemble.  Note that your UUID will differ; you'll have to change the filename in the cell below if you want to get an animation instead of an error message.

In [ ]:
ensfile = open('cd4128e8-8ab1-11e3-9f4f-90e2ba0993e2.ensemble', 'rb')
runslist = pickle.load(ensfile)

fname = starlab.animate_run(runslist[8])
from IPython.display import HTML
video = open(fname, "rb").read()
video_encoded = video.encode("base64")
video_tag = '<video controls alt="test" src="data:video/x-m4v;base64,{0}">'.format(video_encoded)
HTML(data=video_tag)

After you have a good collection of data, we can talk about how to analyze it a little better.

### Now you try it.

You will want to change the number of stars, the number of runs in the ensemble, and perhaps other things.  For the sake of documentation, you'll probably want to make a separate section for each ensemble or set of ensembles you generate.  Write notes in markdown cells (like this one) to remind yourself what you're doing, and then generate the ensembles. 

In [ ]:
rc = Client()
len(rc.ids)

In [ ]:
lbv = rc.load_balanced_view()

In [ ]:
run_ensemble(rc,nstars=1500, nruns=80)

In [ ]:
run_ensemble(rc, nstars=213, nruns=100, runlength=75, kingmodel=False)